In [1]:
import pandas as pd

In [2]:
def primera_llamada(name):

    df = pd.read_csv('ibex35.csv')

    name = df[df['empresa'] == name]

    name = name [['date','open','high','low','close','adj close']]

    name['date'] = pd.to_datetime(name['date'], format="%Y-%m-%d")

    name.sort_values(by='date',ascending= True, inplace=True)

    fecha_inicio = '2021-01-01'
    fecha_fin = '2024-03-01'

    name = pd.DataFrame(name[(name['date'] >= fecha_inicio) & (name['date'] <= fecha_fin)]).reset_index(drop=True)

    return name

In [3]:
acciona = primera_llamada('Acciona')

acciona

,date,open,high,low,close,adj close
0,2021-01-04,117.199997,118.099998,116.699997,116.699997,107.314598
1,2021-01-05,117.599998,117.900002,116.099998,116.599998,107.222633
2,2021-01-06,117.099998,119.400002,116.500000,119.400002,109.797455
3,2021-01-07,119.699997,126.199997,119.699997,125.900002,115.774712
4,2021-01-08,127.000000,128.600006,127.000000,127.800003,117.521904
...,...,...,...,...,...,...
806,2024-02-26,104.449997,104.449997,101.550003,101.550003,101.550003
807,2024-02-27,101.349998,104.250000,101.150002,104.199997,104.199997
808,2024-02-28,104.150002,104.150002,100.000000,100.300003,100.300003
809,2024-02-29,100.500000,104.050003,100.500000,104.050003,104.050003


In [4]:

def preparar_modelo(DF):


    DF_filt = DF[['date','close']]

    DF_filt.close = DF.close.round(2)

    DF_filt= DF_filt.reset_index(drop=True)

    new_df = pd.DataFrame(columns=['d-4', 'd-3', 'd-2', 'd-1', 'do'])

    for i in range(len(new_df.columns)):
        new_df[new_df.columns[i]] = DF_filt['close'].shift(-i)

    new_df['date']= DF_filt.date

    new_df.insert(0, 'date', new_df.pop('date'))

    new_df.dropna(axis=0,inplace=True)

    return new_df



In [5]:
new_df = preparar_modelo(acciona)

new_df

C:\Users\javie\AppData\Local\Temp\ipykernel_30860\4040868792.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF_filt.close = DF.close.round(2)


,date,d-4,d-3,d-2,d-1,do
0,2021-01-04,116.70,116.60,119.40,125.90,127.80
1,2021-01-05,116.60,119.40,125.90,127.80,124.70
2,2021-01-06,119.40,125.90,127.80,124.70,124.80
3,2021-01-07,125.90,127.80,124.70,124.80,126.60
4,2021-01-08,127.80,124.70,124.80,126.60,126.60
...,...,...,...,...,...,...
802,2024-02-20,108.55,108.30,106.70,104.75,101.55
803,2024-02-21,108.30,106.70,104.75,101.55,104.20
804,2024-02-22,106.70,104.75,101.55,104.20,100.30
805,2024-02-23,104.75,101.55,104.20,100.30,104.05


In [6]:

def modelo_train(new_df):

    # Importar herramientas:

    from sklearn.model_selection import train_test_split as tts 
    from sklearn.ensemble import RandomForestRegressor as RFR 
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP
    from sklearn.metrics import mean_squared_error as mse

    # Separar X e y:

    X = new_df.drop(['do','date'],axis=1)
    y = new_df.do

    # Ejecutar tts:

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    X_train.shape, X_test.shape, y_train.shape, y_test.shape

    # Iniciar Modelos

    rfr=RFR()
    omp= OMP()

    # Entrenar

    rfr.fit(X_train, y_train)
    omp.fit(X_train, y_train)
     
    # Predecir

    y_pred_rfr=rfr.predict(X_test)
    y_pred_omp=omp.predict(X_test)

    # Mostrar rmse de cada modelo   
   
    return 'RMSE RFR ======',mse(y_test, y_pred_rfr, squared=False),'REMS OMP ======',mse(y_test, y_pred_omp, squared=False)

In [7]:
modelo_train(new_df)

('RMSE RFR ======', 3.318150097718257, 'REMS OMP ======', 3.049054779178728)

In [8]:
def model(new_df):

    fecha_inicio = '2024-01-01'
    fecha_fin = '2024-02-26'

    # Filtrar el DataFrame
    mask = (new_df['date'] >= fecha_inicio) & (new_df['date'] <= fecha_fin)
    df_pred = new_df.loc[mask]

    df_pred_1 = df_pred[['d-4','d-3','d-2','d-1']]
    
    
    from sklearn.linear_model import OrthogonalMatchingPursuit as OMP

    omp= OMP()

    predicciones_y = []

    # Iterar sobre las 40 posiciones
    for i in range(40):
        # Seleccionar la porción correspondiente de X e y para esta iteración
        X_iteracion = new_df.drop(['date', 'do'], axis=1)[:-40+i]
        y_iteracion = new_df.do[:-40+i]

        # Ajustar el modelo
        omp.fit(X_iteracion, y_iteracion)

        # Seleccionar la fila correspondiente de df_pred para la predicción
        pred = df_pred_1[i:i+1]

        # Realizar la predicción y almacenarla en la lista
        y_pred_omp = omp.predict(pred)
        predicciones_y.append(y_pred_omp[0])   

    pred = pd.DataFrame(predicciones_y)

    pred.columns = ['prediccion']

    df_pred = df_pred.reset_index(drop=True)
    
    df_pred['pred'] = pred.prediccion.round(2)

    df_pred['error'] = df_pred.pred.round(2) - df_pred.do  

    return  df_pred

In [9]:
df_pred = model(new_df)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,131.25,127.15,127.65,126.80,126.30,127.01,0.71
1,2024-01-03,127.15,127.65,126.80,126.30,127.20,126.51,-0.69
2,2024-01-04,127.65,126.80,126.30,127.20,128.80,127.41,-1.39
3,2024-01-05,126.80,126.30,127.20,128.80,128.10,129.00,0.90
4,2024-01-08,126.30,127.20,128.80,128.10,130.80,128.30,-2.50
5,2024-01-09,127.20,128.80,128.10,130.80,127.85,130.99,3.14
6,2024-01-10,128.80,128.10,130.80,127.85,122.35,128.05,5.70
7,2024-01-11,128.10,130.80,127.85,122.35,116.80,122.57,5.77
8,2024-01-12,130.80,127.85,122.35,116.80,117.60,117.03,-0.57
9,2024-01-15,127.85,122.35,116.80,117.60,116.15,117.83,1.68


In [10]:
df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error
0,2024-01-02,131.25,127.15,127.65,126.80,126.30,127.01,0.71
1,2024-01-03,127.15,127.65,126.80,126.30,127.20,126.51,-0.69
2,2024-01-04,127.65,126.80,126.30,127.20,128.80,127.41,-1.39
3,2024-01-05,126.80,126.30,127.20,128.80,128.10,129.00,0.90
4,2024-01-08,126.30,127.20,128.80,128.10,130.80,128.30,-2.50
5,2024-01-09,127.20,128.80,128.10,130.80,127.85,130.99,3.14
6,2024-01-10,128.80,128.10,130.80,127.85,122.35,128.05,5.70
7,2024-01-11,128.10,130.80,127.85,122.35,116.80,122.57,5.77
8,2024-01-12,130.80,127.85,122.35,116.80,117.60,117.03,-0.57
9,2024-01-15,127.85,122.35,116.80,117.60,116.15,117.83,1.68


In [11]:
def escenario(x):

    import math

    df_pred['stock'] = (x/df_pred.do).apply(math.floor)

    df_pred['day_gap'] = df_pred.do.diff()

    df_pred['money'] = df_pred.stock*df_pred.day_gap

    df_pred['evolution'] = x + df_pred['money'].iloc[1:].cumsum().round(2)

    max_money_index = df_pred['money'].idxmax()
    
    df_pred['opti_real'] = x + df_pred['money'].iloc[max_money_index:].cumsum().round(2)

    min_money_index = df_pred['pred'].idxmin()
    
    df_pred['opti_pred'] = x + df_pred['money'].iloc[min_money_index:].cumsum().round(2)

    return df_pred

In [16]:
df_pred = escenario(300)

df_pred

,date,d-4,d-3,d-2,d-1,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred,name
0,2024-01-02,131.25,127.15,127.65,126.80,126.30,127.01,0.71,2,NaN,NaN,NaN,NaN,NaN,acciona
1,2024-01-03,127.15,127.65,126.80,126.30,127.20,126.51,-0.69,2,0.90,1.8,301.8,NaN,NaN,acciona
2,2024-01-04,127.65,126.80,126.30,127.20,128.80,127.41,-1.39,2,1.60,3.2,305.0,NaN,NaN,acciona
3,2024-01-05,126.80,126.30,127.20,128.80,128.10,129.00,0.90,2,-0.70,-1.4,303.6,NaN,NaN,acciona
4,2024-01-08,126.30,127.20,128.80,128.10,130.80,128.30,-2.50,2,2.70,5.4,309.0,NaN,NaN,acciona
5,2024-01-09,127.20,128.80,128.10,130.80,127.85,130.99,3.14,2,-2.95,-5.9,303.1,NaN,NaN,acciona
6,2024-01-10,128.80,128.10,130.80,127.85,122.35,128.05,5.70,2,-5.50,-11.0,292.1,NaN,NaN,acciona
7,2024-01-11,128.10,130.80,127.85,122.35,116.80,122.57,5.77,2,-5.55,-11.1,281.0,NaN,NaN,acciona
8,2024-01-12,130.80,127.85,122.35,116.80,117.60,117.03,-0.57,2,0.80,1.6,282.6,NaN,NaN,acciona
9,2024-01-15,127.85,122.35,116.80,117.60,116.15,117.83,1.68,2,-1.45,-2.9,279.7,NaN,NaN,acciona


In [17]:
def dafatrame(df_pred,name):
    

    df_pred['name'] = name
    
    df_pred = df_pred[['name','date','do','pred','error','stock','day_gap','money','evolution','opti_real','opti_pred']]

    return df_pred

In [18]:
df= dafatrame(df_pred,'acciona')

df

,name,date,do,pred,error,stock,day_gap,money,evolution,opti_real,opti_pred
0,acciona,2024-01-02,126.30,127.01,0.71,2,NaN,NaN,NaN,NaN,NaN
1,acciona,2024-01-03,127.20,126.51,-0.69,2,0.90,1.8,301.8,NaN,NaN
2,acciona,2024-01-04,128.80,127.41,-1.39,2,1.60,3.2,305.0,NaN,NaN
3,acciona,2024-01-05,128.10,129.00,0.90,2,-0.70,-1.4,303.6,NaN,NaN
4,acciona,2024-01-08,130.80,128.30,-2.50,2,2.70,5.4,309.0,NaN,NaN
5,acciona,2024-01-09,127.85,130.99,3.14,2,-2.95,-5.9,303.1,NaN,NaN
6,acciona,2024-01-10,122.35,128.05,5.70,2,-5.50,-11.0,292.1,NaN,NaN
7,acciona,2024-01-11,116.80,122.57,5.77,2,-5.55,-11.1,281.0,NaN,NaN
8,acciona,2024-01-12,117.60,117.03,-0.57,2,0.80,1.6,282.6,NaN,NaN
9,acciona,2024-01-15,116.15,117.83,1.68,2,-1.45,-2.9,279.7,NaN,NaN


In [19]:
df.to_csv('acciona.csv',index=False)